In [2]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import re

## Scrape the Location Counts

# 1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:

In [23]:
# Starbucks
sb_url = "https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state"
sb_response = requests.get(sb_url)
sb_soup = BeautifulSoup(sb_response.content, "html.parser")
table = sb_soup.find("table")

# Initialize empty list
rows = []

# Loop through rows in table

for row in table.find_all('tr')[1:]:
    # Extract state name from <th>
    state = row.find("th").get_text().strip()

    # Extract all store counts from <td> cells
    cells = row.find_all("td")
    
    stores_2021 = int(cells[0].get_text().strip().replace(',', '') or 0)
    stores_2023 = int(cells[1].get_text().strip().replace(',', '') or 0)
    stores_2024 = int(cells[2].get_text().strip().replace(',', '') or 0)

    # Append data as a dictionary
    rows.append({
        "State": state,
        "2021": stores_2021,
        "2023": stores_2023,
        "2024": stores_2024
    })


# Create DataFrame
sb_df = pd.DataFrame(rows)

sb_df.head()

,State,2021,2023,2024
0,California,3080,2959,3117
1,Texas,1346,1215,1409
2,Florida,844,786,892
3,Washington,741,739,736
4,New York,692,643,715


In [25]:
# Dunkin
dd_url = "https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state"
dd_response = requests.get(dd_url)
dd_soup = BeautifulSoup(dd_response.content, "html.parser")
table = dd_soup.find("table")

# Initialize empty list
rows = []

# Loop through rows in table

for row in table.find_all('tr')[1:]:
    # Extract state name from <th>
    state = row.find("th").get_text().strip()

    # Extract all store counts from <td> cells
    cells = row.find_all("td")
    
    stores_2023 = int(cells[0].get_text().strip().replace(',', '') or 0)
    stores_2024 = int(cells[1].get_text().strip().replace(',', '') or 0)

    # Append data as a dictionary
    rows.append({
        "State": state,
        "2023": stores_2023,
        "2024": stores_2024
    })


# Create DataFrame
dd_df = pd.DataFrame(rows)

dd_df.head()

,State,2023,2024
0,New York,1431,1414
1,Massachusetts,1042,1068
2,Florida,909,883
3,New Jersey,872,866
4,Illinois,711,692


# 2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [30]:
# Convert from wide to long format
sb_long = pd.melt(sb_df, id_vars = ["State"], var_name = "Year", value_name = "Store Count")
sb_long["Store Chain"] = "Starbucks"

# Convert from wide to long format
dd_long = pd.melt(dd_df, id_vars = ["State"], var_name = "Year", value_name = "Store Count")
dd_long["Store Chain"] = "Dunkin' Donuts"

# Merge the two DataFrames into one
coffee = pd.concat([sb_long, dd_long], ignore_index=True)

# Display the first few rows of the merged dataset
coffee.head()



,State,Year,Store Count,Store Chain
0,California,2021,3080,Starbucks
1,Texas,2021,1346,Starbucks
2,Florida,2021,844,Starbucks
3,Washington,2021,741,Starbucks
4,New York,2021,692,Starbucks


## Supplemental Data

# 3. Scrape the state names and populations from this wikipedia page. Merge these data with your coffee dataset.

In [63]:
# Scraping State Population Data
wiki_url = "https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population"
wiki_response = requests.get(wiki_url)  
wiki_soup = BeautifulSoup(wiki_response.content, "html.parser") 

# Find the population table
table = wiki_soup.find("table")

# Initialize list for population data
rows = []

# Extract data from table rows (skip the header)
for row in table.find_all("tr")[1:]:
    cells = row.find_all("td")
    state = cells[2].get_text(strip=True)  
    pop_text = cells[3].get_text(strip=True).replace(',', '')  

    population = int(pop_text) if pop_text.isdigit() else 0 
    
    # Append the data to the list
    rows.append({"State": state, "Population": population})

# Create DataFrame for population data
pop_df = pd.DataFrame(rows)

# Merge population data with coffee dataset
merged_data = pd.merge(coffee, pop_df, on="State", how="inner")

# Display the merged data
merged_data


,State,Year,Store Count,Store Chain,Population
0,California,2021,3080,Starbucks,39538223
1,Texas,2021,1346,Starbucks,30145505
2,Florida,2021,844,Starbucks,21538187
3,Washington,2021,741,Starbucks,7705281
4,New York,2021,692,Starbucks,20201249
...,...,...,...,...,...
250,Montana,2024,0,Dunkin' Donuts,1084225
251,North Dakota,2024,0,Dunkin' Donuts,779094
252,Oregon,2024,0,Dunkin' Donuts,4237256
253,South Dakota,2024,0,Dunkin' Donuts,886667


# 4. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these). Merge these values into your big dataset. Note: these values may be repeated for each state.

In [64]:
# Stock prices today
sb_stock = 98.02
dd_stock = 106.48

# Assign stock prices based on Store Chain
merged_data["Stock Price"] = merged_data["Store Chain"].apply(
    lambda x: sb_stock if x == "Starbucks" else dd_stock
)

merged_data

,State,Year,Store Count,Store Chain,Population,Stock Price
0,California,2021,3080,Starbucks,39538223,98.02
1,Texas,2021,1346,Starbucks,30145505,98.02
2,Florida,2021,844,Starbucks,21538187,98.02
3,Washington,2021,741,Starbucks,7705281,98.02
4,New York,2021,692,Starbucks,20201249,98.02
...,...,...,...,...,...,...
250,Montana,2024,0,Dunkin' Donuts,1084225,106.48
251,North Dakota,2024,0,Dunkin' Donuts,779094,106.48
252,Oregon,2024,0,Dunkin' Donuts,4237256,106.48
253,South Dakota,2024,0,Dunkin' Donuts,886667,106.48


# 5. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [67]:
Northeast = [
    "Connecticut", "Maine", "Massachusetts", 
    "New Hampshire", "Rhode Island", "Vermont", 
    "New Jersey", "New York", "Pennsylvania"
]

Midwest = [
    "Illinois", "Indiana", "Iowa", "Kansas", "Michigan", 
    "Minnesota", "Missouri", "Nebraska", "North Dakota", 
    "Ohio", "South Dakota", "Wisconsin"
]

South = [
    "Alabama", "Arkansas", "Delaware", "Florida", "Georgia", 
    "Kentucky", "Louisiana", "Maryland", "Mississippi", 
    "North Carolina", "Oklahoma", "South Carolina", 
    "Tennessee", "Texas", "Virginia", "West Virginia", 
    "District of Columbia"
]

West = [
    "Alaska", "Arizona", "California", "Colorado", "Hawaii", 
    "Idaho", "Montana", "Nevada", "New Mexico", "Oregon", 
    "Utah", "Washington", "Wyoming"
]

# Function to assign state
def state_to_region(state):
    if state in Northeast:
        return "Northeast"

    elif state in Midwest:
        return "Midwest"

    elif state in South:
        return "South"
    
    elif state in West:
        return "West"

# Create Region column
merged_data["Region"] = merged_data["State"].apply(state_to_region)

merged_data


,State,Year,Store Count,Store Chain,Population,Stock Price,Region
0,California,2021,3080,Starbucks,39538223,98.02,West
1,Texas,2021,1346,Starbucks,30145505,98.02,South
2,Florida,2021,844,Starbucks,21538187,98.02,South
3,Washington,2021,741,Starbucks,7705281,98.02,West
4,New York,2021,692,Starbucks,20201249,98.02,Northeast
...,...,...,...,...,...,...,...
250,Montana,2024,0,Dunkin' Donuts,1084225,106.48,West
251,North Dakota,2024,0,Dunkin' Donuts,779094,106.48,Midwest
252,Oregon,2024,0,Dunkin' Donuts,4237256,106.48,West
253,South Dakota,2024,0,Dunkin' Donuts,886667,106.48,Midwest


## Analyze

# 7. Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):
Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.

How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.

Does the distribution of each chain’s stores match population distribution, by both state/region?

Do the financial data match what you’d expect based on the number and locations of the stores? Why or why not?

2. How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.

The financial metric I chose does not change by state or region because it does not change based on the state or region. The stock price shows the market evaluation of the company.

How has the store growth for Starbucks and Dunkin' Donuts differed over the years?

In [80]:
# Filter for Starbucks 
sb_data = merged_data[merged_data["Store Chain"] == "Starbucks"]

# Calculate the change in store count between 2023 and 2024
sb_data["Store_Growth"] = sb_data.groupby("State")["Store Count"].diff()

# Display the growth values
sb_data[["State", "Year", "Store Count", "Store_Growth"]].dropna()

             State  Year  Store Count  Store_Growth
83         Alabama  2023           99          14.0
134        Alabama  2024            0         -99.0
90          Alaska  2023           49           0.0
141         Alaska  2024            0         -49.0
57         Arizona  2023          535         -13.0
..             ...   ...          ...           ...
149  West Virginia  2024            0         -64.0
75       Wisconsin  2023          176          31.0
126      Wisconsin  2024            0        -176.0
99         Wyoming  2023           26           3.0
150        Wyoming  2024            0         -26.0

[102 rows x 4 columns]
             State  Year  Store Count  Store_Growth
83         Alabama  2023           99          14.0
134        Alabama  2024            0         -99.0
90          Alaska  2023           49           0.0
141         Alaska  2024            0         -49.0
57         Arizona  2023          535         -13.0
..             ...   ...          ...   

In [81]:
# Filter for Dunkin' 
dd_data = merged_data[merged_data["Store Chain"] == "Dunkin' Donuts"]

# Calculate the change in store count between 2023 and 2024
dd_data["Store_Growth"] = dd_data.groupby("State")["Store Count"].diff()

# Display the growth values
dd_data[["State", "Year", "Store Count", "Store_Growth"]].dropna()

<positron-console-cell-81>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,State,Year,Store Count,Store_Growth
204,New York,2024,1414,-17.0
205,Massachusetts,2024,1068,26.0
206,Florida,2024,883,-26.0
207,New Jersey,2024,866,-6.0
208,Illinois,2024,692,-19.0
209,Pennsylvania,2024,629,-21.0
210,Connecticut,2024,480,4.0
211,Maryland,2024,303,-8.0
212,Georgia,2024,265,-22.0
213,Ohio,2024,245,-16.0


It seems that the amount of stores for both Starbucks and Dunkin' Donuts has been mostly decreasing from 2023 to 2024. This may reflect closing underperforming stores or responding to economic challenges.

## Automate
Convert your code for Exercises 1-3 above to a function that takes a single argument: the URL. This function should

Scrape the information on state names and corresponding number of store locations on the webpage specified (assume the page has a table in the same form and placement as the ones you scraped above)

Extract the name of the company from either the URL specified or the webpage (assume the URL will have the same format as the ones used above)

Return a clean, organized and tidy dataset. Find a page other than Starbucks and Dunkin’ Donuts to test this on to confirm that it works. It’s fine if this is not related to coffee.

In [107]:
def webscrape(url):
    """
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table")

    # Extract the company name from the URL using regex
    company_name_match = re.search(r'rankings/([\w-]+)-by-state', url)
    company_name = (
        company_name_match.group(1).replace('-', ' ').title()
        if company_name_match else "Unknown Company"
    )

    # Initialize an empty list to store the rows
    rows = []

    # Loop through the rows in the table (skipping the header)
    for row in table.find_all("tr")[1:]:
        # Extract the state name from <th>
        state = row.find("th").get_text().strip()

        # Extract store count from the <td> cell(s)
        store_count = sum(
            int(cell.get_text().strip().replace(',', '') or 0)
            for cell in row.find_all("td")
        )

        # Append the row as a dictionary
        rows.append({"State": state, "Store Count": store_count})

    # Create a DataFrame
    df = pd.DataFrame(rows)

    # Add a column for the company name
    df['Company'] = company_name

    return df

In [109]:
url = "https://worldpopulationreview.com/state-rankings/kroger-by-state"
kroger = webscrape(url)

kroger

,State,Store Count,Company
0,California,302,Kroger
1,Texas,218,Kroger
2,Ohio,211,Kroger
3,Georgia,167,Kroger
4,North Carolina,159,Kroger
5,Indiana,152,Kroger
6,Colorado,148,Kroger
7,Washington,128,Kroger
8,Arizona,126,Kroger
9,Tennessee,121,Kroger


In [112]:
url = "https://worldpopulationreview.com/state-rankings/aldi-by-state"
aldi = webscrape(url)

aldi

,State,Store Count,Company
0,Florida,424,Aldi
1,Illinois,427,Aldi
2,Ohio,318,Aldi
3,Pennsylvania,307,Aldi
4,New York,256,Aldi
5,Texas,254,Aldi
6,Michigan,212,Aldi
7,California,200,Aldi
8,Missouri,193,Aldi
9,North Carolina,185,Aldi
